In [4]:
# Set the PySpark environment variables
import os
os.environ['SPARK_HOME'] = "/Users/venkatasaisabbineni/Work/Spark"
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrame-Operations").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/12 11:46:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data_file_path = "./data/customer_data.csv"
df = spark.read.csv(data_file_path,header = True, inferSchema = True)

In [7]:
df.printSchema()
df.show()
df.count()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Credit_Card_Number: string (nullable = true)
 |-- Credit_Card_Type: string (nullable = true)

+---+----------------+--------------------+------------------+--------------------+
| ID|            Name|               Email|Credit_Card_Number|    Credit_Card_Type|
+---+----------------+--------------------+------------------+--------------------+
|  1|      Rodi Lotze|  rlotze0@netlog.com|               nan|      china-unionpay|
|  2| Vachel Waistell|vwaistell1@answer...|               nan|diners-club-inter...|
|  3|  Mikey Buttrick| mbuttrick2@narod.ru|   5.893663168e+17|             maestro|
|  4|            None|mking3@networkadv...|3539133178799873.0|                 jcb|
|  5| Beverie Ternott|bternott4@rakuten...|3528122067307396.0|                 jcb|
|  6|Adan Spain-Gower|aspaingower5@diig...|3552301123264739.0|                 jcb|
|  7|  Julius Isworth| jisworth6

1000

In [8]:
import pandas as pd
df = pd.read_csv('./data/customer_data.csv')
rows_per_file = 100

num_files = len(df) // rows_per_file

for i in range(num_files):
    start_row = i * rows_per_file
    end_row = (i + 1) * rows_per_file
    chunk = df.iloc[start_row:end_row]
    chunk.to_csv(f'your_file_part_{i+1}.csv', index=False)

# If there are remaining rows, save them as well
if len(df) % rows_per_file != 0:
    start_row = num_files * rows_per_file
    chunk = df.iloc[start_row:]
    chunk.to_csv(f'your_file_part_{num_files+1}.csv', index=False)

In [9]:
selected_columns = df.select("id","name","price")
print("Selected Columns : ")
selected_columns.show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `price` cannot be resolved. Did you mean one of the following? [`ID`, `Name`, `Email`, `Credit_Card_Type`, `Credit_Card_Number`].;
'Project [id#17, name#18, 'price]
+- Relation [ID#17,Name#18,Email#19,Credit_Card_Number#20,Credit_Card_Type#21] csv


In [6]:
filtered_data = df.filter(df.quantity > 20)
print("Filtered Data : ",filtered_data.count())
filtered_data.show()

Filtered Data :  12
+---+--------------+-----------+--------+-----+
| id|          name|   category|quantity|price|
+---+--------------+-----------+--------+-----+
|  6|    Nike Shoes|   Clothing|      30|99.99|
|  7|  Adidas Shoes|   Clothing|      25|89.99|
| 12|        Apples|       Food|     100|  0.5|
| 13|       Bananas|       Food|     150| 0.25|
| 14|       Oranges|       Food|     120| 0.75|
| 15|Chicken Breast|       Food|      50| 3.99|
| 16| Salmon Fillet|       Food|      30| 5.99|
| 24|    Laptop Bag|Accessories|      25|29.99|
| 25|      Backpack|Accessories|      30|24.99|
| 28|         Jeans|   Clothing|      30|59.99|
| 29|       T-shirt|   Clothing|      50|14.99|
| 30|      Sneakers|   Clothing|      40|79.99|
+---+--------------+-----------+--------+-----+



In [7]:
grouped_data = df.groupBy("category").agg({"quantity":"sum","price":"avg"})
print("Grouped & Aggregated Data : ")
grouped_data.show()

Grouped & Aggregated Data : 
+-----------+-------------+------------------+
|   category|sum(quantity)|        avg(price)|
+-----------+-------------+------------------+
|       Food|          450|2.2960000000000003|
|     Sports|           35|             34.99|
|Electronics|           98| 586.6566666666665|
|   Clothing|          200|  99.2757142857143|
|  Furniture|           41|            141.99|
|Accessories|           55|             27.49|
+-----------+-------------+------------------+



In [8]:
df2 = df.select("id","category").limit(15)
joined_data = df.join(df2, "id", "inner")
print("Joined Data : ")
joined_data.show()

Joined Data : 
+---+----------------+-----------+--------+-------+-----------+
| id|            name|   category|quantity|  price|   category|
+---+----------------+-----------+--------+-------+-----------+
|  1|          iPhone|Electronics|      10| 899.99|Electronics|
|  2|         Macbook|Electronics|       5|1299.99|Electronics|
|  3|            iPad|Electronics|      15| 499.99|Electronics|
|  4|      Samsung TV|Electronics|       8| 799.99|Electronics|
|  5|           LG TV|Electronics|      10| 699.99|Electronics|
|  6|      Nike Shoes|   Clothing|      30|  99.99|   Clothing|
|  7|    Adidas Shoes|   Clothing|      25|  89.99|   Clothing|
|  8| Sony Headphones|Electronics|      12| 149.99|Electronics|
|  9|Beats Headphones|Electronics|      20| 199.99|Electronics|
| 10|    Dining Table|  Furniture|      10| 249.99|  Furniture|
| 11|      Study Desk|  Furniture|       8| 149.99|  Furniture|
| 12|          Apples|       Food|     100|    0.5|       Food|
| 13|         Bananas|   

In [17]:
sorted_data = df.orderBy("price")
print("Sorted Data : ")
sorted_data.show()

Sorted Data : 
+---+---------------+-----------+--------+------+
| id|           name|   category|quantity| price|
+---+---------------+-----------+--------+------+
| 13|        Bananas|       Food|     150|  0.25|
| 12|         Apples|       Food|     100|   0.5|
| 14|        Oranges|       Food|     120|  0.75|
| 15| Chicken Breast|       Food|      50|  3.99|
| 16|  Salmon Fillet|       Food|      30|  5.99|
| 29|        T-shirt|   Clothing|      50| 14.99|
| 19|       Yoga Mat|     Sports|      20| 19.99|
| 25|       Backpack|Accessories|      30| 24.99|
| 24|     Laptop Bag|Accessories|      25| 29.99|
| 20|   Dumbbell Set|     Sports|      15| 49.99|
| 28|          Jeans|   Clothing|      30| 59.99|
| 22|   Office Chair|  Furniture|       8| 79.99|
| 30|       Sneakers|   Clothing|      40| 79.99|
|  7|   Adidas Shoes|   Clothing|      25| 89.99|
|  6|     Nike Shoes|   Clothing|      30| 99.99|
| 23|      Bookshelf|  Furniture|      10| 99.99|
| 21|   Coffee Table|  Furniture|  

In [22]:
from pyspark.sql.functions import col,desc,asc
sorted_data = df.orderBy(col("price").desc(),col("quantity").asc())
print("Sorted Data Descending & Ascending : ")
sorted_data.show()

Sorted Data Descending & Ascending : 
+---+----------------+-----------+--------+-------+
| id|            name|   category|quantity|  price|
+---+----------------+-----------+--------+-------+
|  2|         Macbook|Electronics|       5|1299.99|
|  1|          iPhone|Electronics|      10| 899.99|
|  4|      Samsung TV|Electronics|       8| 799.99|
|  5|           LG TV|Electronics|      10| 699.99|
| 26|          Camera|Electronics|      10| 599.99|
|  3|            iPad|Electronics|      15| 499.99|
| 10|    Dining Table|  Furniture|      10| 249.99|
| 17|  Leather Jacket|   Clothing|      15| 199.99|
|  9|Beats Headphones|Electronics|      20| 199.99|
| 11|      Study Desk|  Furniture|       8| 149.99|
| 18|     Winter Coat|   Clothing|      10| 149.99|
|  8| Sony Headphones|Electronics|      12| 149.99|
| 21|    Coffee Table|  Furniture|       5| 129.99|
| 27|         Printer|Electronics|       8| 129.99|
| 23|       Bookshelf|  Furniture|      10|  99.99|
|  6|      Nike Shoes|   C

In [29]:
distinct_rows = df.select("category").distinct()
print("Distinct Product Categories : ")
distinct_rows.show()

Distinct Product Categories : 
+-----------+
|   category|
+-----------+
|       Food|
|     Sports|
|Electronics|
|   Clothing|
|  Furniture|
|Accessories|
+-----------+



In [30]:
dropped_column = df.drop("quantity","category")
print("Dropped Columns : ")
dropped_column.show()
print("Original DataFrame : ")
df.show()

Dropped Columns : 
+---+----------------+-------+
| id|            name|  price|
+---+----------------+-------+
|  1|          iPhone| 899.99|
|  2|         Macbook|1299.99|
|  3|            iPad| 499.99|
|  4|      Samsung TV| 799.99|
|  5|           LG TV| 699.99|
|  6|      Nike Shoes|  99.99|
|  7|    Adidas Shoes|  89.99|
|  8| Sony Headphones| 149.99|
|  9|Beats Headphones| 199.99|
| 10|    Dining Table| 249.99|
| 11|      Study Desk| 149.99|
| 12|          Apples|    0.5|
| 13|         Bananas|   0.25|
| 14|         Oranges|   0.75|
| 15|  Chicken Breast|   3.99|
| 16|   Salmon Fillet|   5.99|
| 17|  Leather Jacket| 199.99|
| 18|     Winter Coat| 149.99|
| 19|        Yoga Mat|  19.99|
| 20|    Dumbbell Set|  49.99|
+---+----------------+-------+
only showing top 20 rows

Original DataFrame : 
+---+----------------+-----------+--------+-------+
| id|            name|   category|quantity|  price|
+---+----------------+-----------+--------+-------+
|  1|          iPhone|Electronics

In [31]:
df_with_new_column = df.withColumn("revenue",df.quantity * df.price)
print("DataFrame with New Column : ")
df_with_new_column.show()

DataFrame with New Column : 
+---+----------------+-----------+--------+-------+------------------+
| id|            name|   category|quantity|  price|           revenue|
+---+----------------+-----------+--------+-------+------------------+
|  1|          iPhone|Electronics|      10| 899.99|            8999.9|
|  2|         Macbook|Electronics|       5|1299.99|           6499.95|
|  3|            iPad|Electronics|      15| 499.99|           7499.85|
|  4|      Samsung TV|Electronics|       8| 799.99|           6399.92|
|  5|           LG TV|Electronics|      10| 699.99|            6999.9|
|  6|      Nike Shoes|   Clothing|      30|  99.99|            2999.7|
|  7|    Adidas Shoes|   Clothing|      25|  89.99|           2249.75|
|  8| Sony Headphones|Electronics|      12| 149.99|           1799.88|
|  9|Beats Headphones|Electronics|      20| 199.99|            3999.8|
| 10|    Dining Table|  Furniture|      10| 249.99|            2499.9|
| 11|      Study Desk|  Furniture|       8| 149.

In [32]:
df_with_alias = df.withColumnRenamed("price","product_price")
print("DataFrame with Aliased Column : ")
df_with_alias.show()

DataFrame with Aliased Column : 
+---+----------------+-----------+--------+-------------+
| id|            name|   category|quantity|product_price|
+---+----------------+-----------+--------+-------------+
|  1|          iPhone|Electronics|      10|       899.99|
|  2|         Macbook|Electronics|       5|      1299.99|
|  3|            iPad|Electronics|      15|       499.99|
|  4|      Samsung TV|Electronics|       8|       799.99|
|  5|           LG TV|Electronics|      10|       699.99|
|  6|      Nike Shoes|   Clothing|      30|        99.99|
|  7|    Adidas Shoes|   Clothing|      25|        89.99|
|  8| Sony Headphones|Electronics|      12|       149.99|
|  9|Beats Headphones|Electronics|      20|       199.99|
| 10|    Dining Table|  Furniture|      10|       249.99|
| 11|      Study Desk|  Furniture|       8|       149.99|
| 12|          Apples|       Food|     100|          0.5|
| 13|         Bananas|       Food|     150|         0.25|
| 14|         Oranges|       Food|     

In [33]:
spark.stop()